# Libraries declaration

In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium
import gtfs_kit as gk
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import folium
from pathlib import Path
import time
import numpy as np

General configurations

In [2]:
# Ensure the OSMnx settings are optimized for your needs
ox.settings.use_cache = True
ox.settings.log_console = True


# Importing all the GTFS files

To save on the memory used, we don't import empty columns and use more efficient data types when possible.

In [3]:
dir_GTFS= "/home/lubuntu/GSDMA_2024/Tec GTFS"
print("Loading stops")
stops = pd.read_csv(dir_GTFS+"/stops.txt", usecols=['stop_id','stop_name','stop_lat','stop_lon','zone_id','location_type'])
print("Loading stop_times")
stop_times = pd.read_csv(dir_GTFS+"/stop_times.txt", dtype={'service_id':'category',
                                                            'pickup_type':'category',
                                                            'drop_off_type':'category',
                                                            'stop_sequence':'int8',
                                                           'departure_time':'string',
                                                           'arrival_time':'string'}
                        )#,parse_dates=["arrival_time", "departure_time"], date_format="%H:%M")
stop_times['arrival_time']=pd.to_timedelta(stop_times['arrival_time'])
stop_times['departure_time']=pd.to_timedelta(stop_times['departure_time'])

print("Loading trips")
trips = pd.read_csv(dir_GTFS+"/trips.txt", dtype={'service_id':'category',
                                                  'trip_short_name':'category',
                                                  'direction_id':'int8'})
print("Loading shapes")
shapes = pd.read_csv(dir_GTFS+"/shapes.txt", dtype={'shape_id':'category',
                                                    'shape_pt_sequence':'uint32'})
print("Loading routes")
routes = pd.read_csv(dir_GTFS+"/routes.txt", usecols=['route_id','agency_id','route_short_name','route_long_name','route_type'],
                    dtype={'route_type':'uint8',
                          'agency_id':'category'})
#print("Loading agency")
#agency = pd.read_csv(dir_GTFS+"/agency.txt")
print("Loading calendar")
calendar = pd.read_csv(dir_GTFS+"/calendar.txt",dtype={'monday':'boolean',
                                                      'tuesday':'boolean',
                                                      'wednesday':'boolean',
                                                      'thursday':'boolean',
                                                      'friday':'boolean',
                                                      'saturday':'boolean',
                                                      'sunday':'boolean'},
                      parse_dates=["start_date", "end_date"], date_format="%Y%m%d")
#print("Loading calendar_dates")
#calendar_dates = pd.read_csv(dir_GTFS+"/calendar_dates.txt", dtype={'exception_type':'uint8'}, parse_dates=["date"], date_format="%Y%m%d")

Loading stops
Loading stop_times
Loading trips
Loading shapes
Loading routes
Loading calendar


Some buses arrival and departure times are above 24 hours (Ex: 24:03:00). They indicate that the bus trip started the previous day and is still considered as active even if it is not today.
To handle those times, the columns 'departure_time' and 'arrival_time' are not stored as datetime but as timedelta.

In [4]:
stop_times[stop_times['arrival_time']>pd.to_timedelta('24:00:00')]#7645 stops after midnight

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
466938,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:03:00,1 days 00:03:00,LaAbush*,1,0,0
466939,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:06:00,1 days 00:06:00,LaAelis2,2,0,0
466940,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:07:00,1 days 00:07:00,LaApost2,3,0,0
466941,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:09:00,1 days 00:09:00,LaAmise2,4,0,0
466942,41591058-L_PA_2024-24_SP_VA-Sem-Vac-30-0000100,1 days 00:11:00,1 days 00:11:00,LaAhaup2,5,0,0
...,...,...,...,...,...,...,...
5251846,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:08:00,1 days 00:08:00,Lheloti1,32,0,0
5251847,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:09:00,1 days 00:09:00,Lhehoux1,33,0,0
5251848,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:09:00,1 days 00:09:00,Lheneuv1,34,0,0
5251849,45163517-L_PA_2024-24_LG_DI-Dimanche-11,1 days 00:10:00,1 days 00:10:00,Lhecarc1,35,0,0


# Converting the dataframes to geodataframes
## Stops
The coordinates of the bus stops are converted to points. The CRS used is the World Geodetic System 1984 as used in GPRS

In [5]:
geometry=gpd.points_from_xy(stops['stop_lon'], stops['stop_lat'], z=None, crs='epsg:4326')
geo_stops=gpd.GeoDataFrame(data=stops, geometry=geometry)
geo_stops=geo_stops.to_crs(3812)
#geo_stops.sample(n=30).plot()
del stops

## Shapes

In [6]:
geometry=gpd.points_from_xy(shapes['shape_pt_lon'], shapes['shape_pt_lat'], z=None, crs='epsg:4326')
geo_shapes=gpd.GeoDataFrame(data=shapes, geometry=geometry)
#geo_shapes.sample(50).plot()
del shapes

In [7]:
#Sort the lines by id and Sequence so they are ordered
geo_shapes_sorted = geo_shapes.sort_values(by=['shape_id', 'shape_pt_sequence'])

#Group the points by 'route'
lines = geo_shapes_sorted.groupby('shape_id', observed=True).apply(lambda x: LineString(x.geometry.tolist()))
lines=lines.reset_index()
lines = gpd.GeoDataFrame(data=lines['shape_id'], geometry=lines[0], crs=geo_shapes.crs)
#lines.sample(20).explore()
del geo_shapes
del geo_shapes_sorted

/tmp/ipykernel_4065/1013893770.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lines = geo_shapes_sorted.groupby('shape_id', observed=True).apply(lambda x: LineString(x.geometry.tolist()))


# Functions

In [8]:
def filter_bus_stops(geo_stops, center_gdf, radius=3000):
    """
   Filter the bus stops within a given radius of the starting point.
    """
    
    # Buffer around the center point (in meters)
    center_buffer = center_gdf.to_crs(epsg=3812).buffer(radius).to_crs(epsg=4326)#4326
    
    # Filter stops within the buffer
    filtered_stops = geo_stops[geo_stops.geometry.within(center_buffer.iloc[0])]
    
    return filtered_stops

In [9]:
# 2. Calculate shortest paths to bus stops
def calculate_shortest_paths_to_stops(graph, start_point, bus_stops):
    """
    Calculate shortest paths from the start point to each bus stop.
    """
    start_node = ox.distance.nearest_nodes(graph, start_point[1], start_point[0])
    paths = []

    for _, stop in bus_stops.iterrows():
        stop_coords = (stop.geometry.y, stop.geometry.x)
        stop_node = ox.distance.nearest_nodes(graph, stop_coords[1], stop_coords[0])
        try:
            shortest_path = nx.shortest_path(graph, source=start_node, target=stop_node, weight="length")
            path_length = sum(nx.get_edge_attributes(graph, "length").get((shortest_path[i], shortest_path[i+1], 0), 0) for i in range(len(shortest_path) - 1))
            paths.append({"stop_id": stop.stop_id, "path": shortest_path, "length": path_length,"stop_point":stop_node})
            #print(f"The length of the shortest path is {path_length:.2f} meters.")
        except nx.NetworkXNoPath:
            # Skip if there's no path
            continue
    
    return paths

In [10]:
# 3. Plot the star graph
def plot_star_graph(graph, start_point, paths, bus_stops):
    """
    Plot the graph with shortest paths to bus stops.
    """
    
    # Initialize a folium map centered at the start point
    m = folium.Map(location=start_point, zoom_start=14)
    
    # Add the graph edges to the map
    for u, v, data in graph.edges(data=True):
        if "geometry" in data:
            coords = [(lat, lon) for lon, lat in data["geometry"].coords]
        else:
            coords = [(graph.nodes[u]["y"], graph.nodes[u]["x"]), (graph.nodes[v]["y"], graph.nodes[v]["x"])]
        folium.PolyLine(coords, color="gray", weight=1).add_to(m)
    
    # Add the start point marker
    folium.Marker(location=start_point, popup="Start", icon=folium.Icon(color="green")).add_to(m)
    
    # Add bus stops and paths
    for path_info in paths:
        stop_id = path_info["stop_id"]
        path = path_info["path"]
        stop = bus_stops[bus_stops.stop_id == stop_id].iloc[0]
        
        # Add path to map
        path_coords = [(graph.nodes[node]["y"], graph.nodes[node]["x"]) for node in path]
        folium.PolyLine(path_coords, color="red", weight=2).add_to(m)
        
        # Add bus stop marker
        stop_coords = (stop.geometry.y, stop.geometry.x)
        folium.Marker(location=stop_coords, popup=f"Stop ID: {stop_id}", icon=folium.Icon(color="blue")).add_to(m)
    
    # Return the map
    return m

In [11]:
def create_gdf_with_paths(graph, start_point, paths, bus_stops):
    """
    Create a GeoDataFrame with the start point, bus stops, and paths.
    """
    # Start point as a geometry
    start_point_geom = Point(start_point[1], start_point[0])
    
    # Create a list of all nodes (start point + bus stops)
    all_nodes = [start_point_geom]
    all_paths = []

    for path_info in paths:
        stop_id = path_info["stop_id"]
        path = path_info["path"]
        stop_point = path_info["stop_point"]  # Correctly access stop_point

        # Create the path geometries (connecting the nodes)
        path_coords = [(graph.nodes[node]["x"], graph.nodes[node]["y"]) for node in path]
        for coords in path_coords:
            all_nodes.append(Point(coords))

        # Add the stop point to the nodes list
        all_nodes.append(stop_point)
    
    # Create a GeoDataFrame with the 'geometry' column
    gdf_nodes = gpd.GeoDataFrame(
        {'geometry': all_nodes}
    )
    
    # Set the CRS to 'EPSG:4326' after geometry is assigned
    gdf_nodes.set_crs("EPSG:4326", allow_override=True, inplace=True)
    
    # Add the type of node (start, path, stop)
    gdf_nodes['type'] = ['start'] + ['path'] * (len(all_nodes) - 2) + ['stop']
    
    return gdf_nodes


In [45]:
def compute_walk_time(A_stop, B_stop):
    walk_speed=4#km/h
    #For the moment, I divide the distance between them by 4km/h 
    #In the future, we will use the length of the shortest walkable path between the points 

    #The stops are projected in the Belgian Lambert 2008 coordinates system (crs=3812) to have accurate distances
    if(type(B_stop)==gpd.geodataframe.GeoDataFrame):
        B_stop=B_stop.head(1)
        A_stop=A_stop.head(1)
    birdfly_dist=A_stop.distance(B_stop, align=False)
    walk_time=birdfly_dist/(1000*walk_speed/60)#walking time in minutes (float)
    #walk_time=walk_time.mean()#If several stops have the same name, take the average of their walking distance
    walk_time=pd.to_timedelta(walk_time,unit='min')#Converted to TimeDelta
    return walk_time

In [18]:
def find_next_stops(best_arrival_time, active_trips, active_stop_times, cur_stop_id, cur_time):
    walk_range=1000#Max length in meters between the current stop and the other stops reached by foot
    
    #walk_time=compute_walk_time(arrival_stop, cur_stop)
    #Extract trips stopping by the current bus stop that are active
    #print(cur_time)
    cur_stop_times=active_stop_times[((active_stop_times['stop_id'].isin(cur_stop_id))&
                                     (active_stop_times['departure_time']>cur_time)&
                                     (active_stop_times['departure_time']<(best_arrival_time))
                                     #(active_stop_times[trip_id].isin(active_trips_id))
                                      )].sort_values('departure_time')
#    cur_stop_times=active_stop_times.query("(stop_id.isin(@cur_stop_id))"
#                                    +"&(departure_time>@cur_time)"
#                                    #+"&(trip_id.isin(@active_trips_id))"
#                                     ).sort_values('departure_time')
    
    #Extract the other stops that can be reached with the trips
    other_stops_times=[]
    for row in cur_stop_times.itertuples(index=False):
        departure_seq=int(row.stop_sequence)
        other_stops_time=active_stop_times[((active_stop_times['trip_id']==row.trip_id)
                                      &(active_stop_times['arrival_time']<(cur_time+walk_time))
                                      &(active_stop_times['stop_sequence']>departure_seq)
                                     )]
        other_stops_times.append(other_stops_time)
        #active_stop_times=active_stop_times[~active_stop_times.eq(other_stops_time,axis=0).all(axis=1)]
        #print(row.departure_time, end='\r')

    #Add the stops reachable by foot
    cur_stop=geo_stops[geo_stops['stop_id'].isin(cur_stop_id)]
    walked_stops=geo_stops[geo_stops.within(cur_stop.buffer(1000).geometry.iloc[0])]
    walked_stop_times=pd.DataFrame(columns=stop_times.columns)
    walked_stop_times['stop_id']=walked_stops['stop_id']
    walked_stop_times['trip_id']='walking'
    walked_stop_times['arrival_time']=cur_time+compute_walk_time(walked_stops, cur_stop.iloc[0].geometry)
    #print(walked_stop_times['arrival_time'])
    walked_stop_times['departure_time']=walked_stop_times['arrival_time']
    walked_stop_times['stop_sequence']=1
    walked_stop_times['pickup_type']=0
    walked_stop_times['drop_off_type']=0
    
    #print(walked_stop_times)
    if(walked_stop_times.size>0):
        other_stops_times.append(walked_stop_times)
    
    if(len(other_stops_times)>0):
        other_stops_times=pd.concat(other_stops_times)
    else:
        print("No next stops found")
        return pd.DataFrame()
    return other_stops_times

In [63]:
def explore_node(geo_stops, active_trips, active_stop_times, found_stop_times, best_arrival_time, best_path, cur_stop_time):
    num_trans=cur_stop_time['number_trips']+1
    #print(cur_stop_time[['stop_id','final_arrival_time', 'intermediary_stops']])
    num_trans=num_trans.iloc[0]
    max_trans=5
    if(num_trans>max_trans):
        print("Max number of transfers reached")
        return pd.DataFrame(), best_arrival_time, best_path
    cur_stop_id=cur_stop_time['stop_id']#pd.Series(row.stop_id)
    cur_time=cur_stop_time['arrival_time'].iloc[0]#row.arrival_time
    
    #Extract the stops directly linked to the current stop
    #print(previous_stops)
    other_stops_times=find_next_stops(best_arrival_time, active_trips, active_stop_times, cur_stop_id, cur_time)

    if(other_stops_times.size>0):
        #print(previous_stops)
        other_stops_times['number_trips']=num_trans
        #print(cur_stop_time)
        other_stops_times['intermediary_stops']=other_stops_times.apply(lambda x:UpdatePath(cur_stop_time,x),axis=1)
        
        #Compute the proximity of the stops discovered from the arrival stop
        other_stops=geo_stops.merge(other_stops_times.reset_index(), on='stop_id',how='right')
        new_walk_times=compute_walk_time(other_stops, arrival_stop.iloc[0].geometry)
        other_stops['final_arrival_time']=other_stops['arrival_time']+new_walk_times
        other_stops=other_stops.set_index('index')
        new_best_arrival_time=other_stops['final_arrival_time'].min()
        print(new_best_arrival_time)
        #print(other_stops)
        if(new_best_arrival_time<best_arrival_time):
            #Update the current best path
            previous_stops=other_stops[other_stops['final_arrival_time']==new_best_arrival_time].copy()
            print(previous_stops)
            best_path=previous_stops['intermediary_stops'].iloc[0]
            best_path.append((previous_stops['stop_id'].iloc[0],previous_stops['trip_id'].iloc[0]))
            best_path.append((arrival_stop['stop_id'].iloc[0],'walking'))
            print(best_path)
            best_arrival_time=new_best_arrival_time

        #TODO: Filter out the stops from which you would arrive later at the arrival stop than the best path even in bus in bird fly
        other_stops_times['final_arrival_time']=other_stops['final_arrival_time']
        other_stops_times=other_stops_times[other_stops_times['arrival_time']<=best_arrival_time]
    else:
        return pd.DataFrame(), best_arrival_time, best_path
    return other_stops_times, best_arrival_time, best_path
def UpdatePath(previous_stop, stop):
    #print(stop)
    cur_stop_id=previous_stop['stop_id']
    #Update the path going to these stops
    previous_stops=previous_stop.loc[:,'intermediary_stops'].iloc[0].copy()#row.intermediary_stops[:]
    previous_stops.append((cur_stop_id.iloc[0], stop['trip_id']))
    return previous_stops#TODO: also include the trips used (or walking)

# Main process

In [76]:
# Def/home/lubuntu/.local/share/jupyter/runtimeine the starting location
departure_date="2024-11-05"
departure_time="08:00:00"
departure_stop_name="MONS Brisselot"#"MONS Place de Flandre"#
arrival_stop_name="SOIGNIES Place du Jeu de Balle"

## Graph of the shortest walkable paths

In [12]:
radius = 5000  # For graph creation
bus_stop_radius = 3000  # For filtering stops
    
# Path to GTFS stops.txt file
gtfs_stops_file = 'Tec GTFS/stops.txt' # Replace with your GTFS file path
    
# Get the center point
start_point = ox.geocode(departure_stop_name)
    # Create a GeoDataFrame for the center point
start_gdf = gpd.GeoDataFrame(
    {"geometry": [Point(start_point[1], start_point[0])]},
    crs="EPSG:4326",
)
# Create the graph
graph = ox.graph_from_point(start_point, dist=radius, network_type="walk")
print("Graph created")
# Filter bus stops within the specified radius
filtered_stops = filter_bus_stops(geo_stops, start_gdf, radius=bus_stop_radius)
print("Bus stops retrieved")
# Calculate shortest paths to bus stops
paths = calculate_shortest_paths_to_stops(graph, start_point, filtered_stops)
print("Shortests paths found")
print("Plotting...")
# Plot the star graph
star_map = plot_star_graph(graph, start_point, paths, filtered_stops)
#star_map.save("star_graph_map.html")
star_map

/home/lubuntu/.local/share/virtualenvs/GSDMA_2024-IfVh60CW/lib/python3.9/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


Graph created


NameError: name 'filter_bus_stops' is not defined

## Shortests bus paths

In [69]:
#Find the bus stops with a given name within 
geo_stops[geo_stops['stop_name'].str.contains('CHARL')].iloc[10:20]

,stop_id,stop_name,stop_lat,stop_lon,zone_id,location_type,geometry
3518,Cchbrou1,CHARLEROI Broucheterre,50.417643,4.444403,7101,0,POINT (655382.082 622977.558)
3519,Cchbrou2,CHARLEROI Broucheterre,50.417618,4.445364,7101,0,POINT (655450.382 622974.856)
3520,Cchcase1,CHARLEROI Casernes,50.409921,4.450268,7101,0,POINT (655799.949 622119.138)
3521,Cchcase2,CHARLEROI Casernes,50.409832,4.450226,7101,0,POINT (655796.976 622109.235)
3522,Cchcase3,CHARLEROI Casernes,50.410976,4.450288,7101,0,POINT (655801.227 622236.487)
3523,Cchcase4,CHARLEROI Casernes,50.410233,4.449966,7101,0,POINT (655778.441 622153.815)
3524,Cchcaya1,CHARLEROI Cayauderie,50.426883,4.451436,7101,0,POINT (655880.627 624005.917)
3525,Cchcaya2,CHARLEROI Cayauderie,50.426901,4.451916,7101,0,POINT (655914.730 624007.962)
3526,Cchccom1,CHARLEROI Centre Commercial (Ville2),50.418608,4.456818,7101,0,POINT (656264.250 623085.980)
3527,Cchccom2,CHARLEROI Centre Commercial (Ville2),50.418436,4.457035,7101,0,POINT (656279.697 623066.869)


In [71]:
walking_range = 1000#The algorithm considers that bus stops closer than that value in meters can be linked by walking

start_time = time.time()
#Extract the stops and the stop_id's
departure_stop=geo_stops.query("stop_name==@departure_stop_name")
arrival_stop=geo_stops.query("stop_name==@arrival_stop_name")

arrival_stop_id=arrival_stop['stop_id']
departure_stop_id=departure_stop['stop_id']

#The walking time between the departure and the arrival is computed
walk_time=compute_walk_time(departure_stop, arrival_stop)
walk_time=walk_time.mean()

departure_time=pd.to_timedelta(departure_time)
best_arrival_time=departure_time+walk_time
print(f"First arrival time: {best_arrival_time}")
best_path=[(departure_stop_id.iloc[0],'Already here'),(arrival_stop_id.iloc[0],'walking')]

#Find the schedule of the day
week_day=pd.Timestamp(departure_date).day_name().lower()
active_services=calendar.query("(start_date<=@departure_date)&(@departure_date<=end_date)")
active_services=active_services[active_services[week_day]]
active_trips=trips[trips['service_id'].isin(active_services['service_id'])]
active_trips_id=active_trips['trip_id']
active_stop_times=stop_times[stop_times['trip_id'].isin(active_trips_id)]
active_stop_times=active_stop_times[active_stop_times['arrival_time']<best_arrival_time]

#Find the stoptimes of the departure stop
found_stop_times=active_stop_times[((active_stop_times['stop_id'].isin(departure_stop_id))&
                                     (active_stop_times['departure_time']>departure_time)&
                                     (active_stop_times['departure_time']<(best_arrival_time))
                                     #(active_stop_times[trip_id].isin(active_trips_id))
                                      )].sort_values('departure_time')
found_stop_times.loc[:, "intermediary_stops"] = [[(departure_stop_id.iloc[0],'Already here')]] * len(found_stop_times)
found_stop_times['final_arrival_time']=best_arrival_time

##Add the stops reachable by foot from the departure point
#meas_geo_stops=geo_stops.to_crs(3812)
#walked_stops=meas_geo_stops[meas_geo_stops.within(start_gdf.to_crs(3812).buffer(walking_range).geometry.iloc[0])]
#walked_stop_times=pd.DataFrame(columns=stop_times.columns)
#walked_stop_times['stop_id']=walked_stops['stop_id']
#walked_stop_times['trip_id']='walking'
#walked_stop_times['arrival_time']=departure_time+compute_walk_time(walked_stops, start_gdf.to_crs(3812).iloc[0].geometry)
#walked_stop_times['departure_time']=walked_stop_times['arrival_time']
#walked_stop_times['stop_sequence']=1
#walked_stop_times['pickup_type']=0
#walked_stop_times['drop_off_type']=0
#walked_stop_times.loc[:, "intermediary_stops"] = [[(departure_stop_id.iloc[0],'walking')]] * len(walked_stop_times) 
#found_stop_times=pd.concat([found_stop_times,walked_stop_times])

found_stop_times['number_trips']=0
found_stop_times['explored']=False
new_stop_times=found_stop_times[found_stop_times['explored']==False]#The found stops not already explored

#other_stops_times=find_next_stops(best_arrival_time, active_trips, active_stop_times, departure_stop_id, departure_time)

#found_stop_times['explored']=True
#other_stops_times['number_trips']=1
#other_stops_times.loc[:, "intermediary_stops"] = [[departure_stop_id.iloc[0]]] * len(other_stops_times) #other_stops_times['intermediary_stops']=''
##TODO: Add other stops within walkable distance
#other_stops_times=other_stops_times.sort_values('arrival_time').drop_duplicates('stop_id')#We keep only the earliest arrival time to a stop.
#other_stops_times['explored']=False
#
##Compare the walking time from the new stops
#other_stops=geo_stops.merge(other_stops_times.reset_index(), on='stop_id',how='right', )
#new_walk_times=compute_walk_time(other_stops.to_crs(3812), arrival_stop.to_crs(3812).iloc[0].geometry)
#other_stops['final_arrival_time']=other_stops['arrival_time']+new_walk_times
#other_stops=other_stops.set_index('index')
#new_best_arrival_time=other_stops['final_arrival_time'].min()
#if(new_best_arrival_time<best_arrival_time):
#    print(f"New best arrival time: {new_best_arrival_time}")
#    best_path=[(departure_stop_id.iloc[0], 'w')]
#    best_arrival_time=new_best_arrival_time
#    #Drop the scheduled stops later than the current best arrival time 
#    active_stop_times=active_stop_times[active_stop_times['arrival_time']<best_arrival_time]

#other_stops_times['final_arrival_time']=other_stops['final_arrival_time']
#found_stop_times=pd.concat([found_stop_times,other_stops_times])
#found_stop_times=found_stop_times.sort_values('arrival_time').drop_duplicates('stop_id')#We keep only the earliest arrival time to a stop.
#found_stop_times=found_stop_times.sort_values('final_arrival_time')

#new_stop_times=found_stop_times[found_stop_times['explored']==False]#The found stops not already explored


while(new_stop_times.size>0):
    new_stop_times=found_stop_times[found_stop_times['explored']==False]#The found stops not already explored
    cur_stop_time=new_stop_times.iloc[0:1,:]

    other_stops_times, new_best_arrival_time, new_best_path=explore_node(geo_stops, active_trips, active_stop_times, found_stop_times, best_arrival_time, best_path, cur_stop_time)
    other_stops_times['explored']=False
    found_stop_times.loc[cur_stop_time.index,'explored']=True

    #Drop the scheduled stops later than the current best arrival time
    if(new_best_arrival_time<best_arrival_time):
        print(f"New best arrival time: {new_best_arrival_time}")
        best_arrival_time=new_best_arrival_time
        best_path=new_best_path
        active_stop_times=active_stop_times[active_stop_times['arrival_time']<=best_arrival_time]
        found_stop_times=found_stop_times[found_stop_times['arrival_time']<=best_arrival_time]

    found_stop_times=pd.concat([found_stop_times,other_stops_times])
    found_stop_times=found_stop_times.sort_values('arrival_time')
    found_stop_times=found_stop_times.drop_duplicates('stop_id')#We keep only the earliest arrival time to a stop.
    found_stop_times=found_stop_times.sort_values('final_arrival_time')
    new_stop_times=found_stop_times[found_stop_times['explored']==False]#The found stops not already explored
    print(f"{found_stop_times.shape[0]-new_stop_times.shape[0]}/{found_stop_times.shape[0]}")
end_time = time.time()
print(f"Best arrival time:{best_arrival_time}")
print(f"Best path:{best_path}")
print(f"It took {end_time-start_time} seconds to compute")

First arrival time: 0 days 16:53:30.233488752
0 days 16:54:30.233488752
1/49
0 days 14:17:17.996712168
          stop_id    stop_name   stop_lat  stop_lon  zone_id  location_type  \
index                                                                         
5347644  H3bi105a  BINCHE SNCB  50.408993  4.172723     3161              0   

                              geometry                            trip_id  \
index                                                                       
5347644  POINT (636071.920 622028.596)  45184920-H_2024-H24_P2-Sem-N-3-11   

           arrival_time  departure_time  stop_sequence pickup_type  \
index                                                                
5347644 0 days 09:14:00 0 days 09:14:00             38           0   

        drop_off_type  number_trips  \
index                                 
5347644             0             2   

                                        intermediary_stops  \
index                               

In [72]:
geo_paths=[]
for i in range(len(best_path)):
    print(i)
    inter=best_path[i]
    inter_stop_id=inter[0]
    inter_stop=geo_stops[geo_stops['stop_id']==inter_stop_id]
    way=inter[1]
    if(way=="walking"):
        if(i==len(best_path)-1):
            previous_stop_id=best_path[i-1][0]
            previous_stop=geo_stops[geo_stops['stop_id']==previous_stop_id]
            inter_shape=gpd.GeoDataFrame(columns=lines.columns, crs=geo_stops.crs)
            inter_shape['shape_id']=previous_stop['stop_id']
            inter_shape['shape_id']='walking'
            inter_shape['walking']=True
            inter_shape['geometry']=LineString([previous_stop.iloc[0].geometry, inter_stop.iloc[0].geometry])
            inter_shape=inter_shape.to_crs(4326)
            print(inter_shape)
        else:
            next_stop_id=best_path[i+1][0]
            next_stop=geo_stops[geo_stops['stop_id']==next_stop_id]
            inter_shape=gpd.GeoDataFrame(columns=lines.columns, crs=geo_stops.crs)
            inter_shape['shape_id']=next_stop['stop_id']
            inter_shape['shape_id']='walking'
            inter_shape['walking']=True
            inter_shape['geometry']=LineString([inter_stop.iloc[0].geometry, next_stop.iloc[0].geometry])
            inter_shape=inter_shape.to_crs(4326)
            #print(inter_shape)
        geo_paths.append(inter_shape)
    elif(way=="Already here"):
        print(end='')
    else:
        inter_trip=trips[trips['trip_id']==way]
        inter_shape_id=inter_trip['shape_id'].iloc[0]
        inter_shape=lines[lines['shape_id']==inter_shape_id]
        inter_shape['walking']=False
        print(inter_shape)
        geo_paths.append(inter_shape)
best_path_gdf=pd.concat(geo_paths)


0
1
2
       shape_id                                           geometry  walking
1328  H10220033  LINESTRING (3.94325 50.45469, 3.94330 50.45473...    False
3
       shape_id                                           geometry  walking
1908  H21360048  LINESTRING (4.20703 50.47847, 4.20715 50.47859...    False
4
5
      shape_id                                           geometry  walking
939  C00M20063  LINESTRING (4.27193 50.40580, 4.27193 50.40580...    False
6
      shape_id                                           geometry  walking
939  C00M20063  LINESTRING (4.27193 50.40580, 4.27193 50.40580...    False
7
     shape_id                                         geometry  walking
5446  walking  LINESTRING (4.45062 50.41718, 4.45682 50.41861)     True


/home/lubuntu/.local/share/virtualenvs/GSDMA_2024-IfVh60CW/lib/python3.9/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/lubuntu/.local/share/virtualenvs/GSDMA_2024-IfVh60CW/lib/python3.9/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/lubuntu/.local/share/virtualenvs/GSDMA_2024-IfVh60CW/lib/python3.9/site-packages/geopandas/geodataframe.py:1528:

In [14]:
lines.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [56]:
geo_stops[geo_stops['stop_id']=='H1ms294c']

,stop_id,stop_name,stop_lat,stop_lon,zone_id,location_type,geometry
7786,H1ms294c,MONS Place de Flandre,50.454292,3.96134,3101,0,POINT (3.96134 50.45429)


In [73]:
found_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,intermediary_stops,final_arrival_time,number_trips,explored
847446,42781360-C2024-choi-Sem-N-3-11,0 days 10:00:00,0 days 10:00:00,CMwate2,18.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 10:07:01.427447796,5.0,True
847445,42781360-C2024-choi-Sem-N-3-11,0 days 09:58:00,0 days 09:58:00,CMpa2,17.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 10:13:04.673792946,5.0,True
847447,42781360-C2024-choi-Sem-N-3-11,0 days 10:02:00,0 days 10:02:00,CMjans1,19.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 10:13:39.065434062,5.0,True
847444,42781360-C2024-choi-Sem-N-3-11,0 days 09:57:00,0 days 09:57:00,CMpige1,16.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 10:18:15.321169248,5.0,True
847448,42781360-C2024-choi-Sem-N-3-11,0 days 10:04:00,0 days 10:04:00,CMparc1,20.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 10:23:06.005910720,5.0,True
...,...,...,...,...,...,...,...,...,...,...,...
6765,walking,0 days 09:43:22.167203208,0 days 09:43:22.167203208,H1do111b,1.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 21:53:53.874901674,5.0,True
6780,walking,0 days 09:44:26.118170478,0 days 09:44:26.118170478,H1do121b,1.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 21:59:37.577825706,5.0,True
6779,walking,0 days 09:46:34.521121386,0 days 09:46:34.521121386,H1do121a,1.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 22:03:04.879021398,5.0,True
8487,walking,0 days 09:51:33.501995460,0 days 09:51:33.501995460,H1wi147a,1.0,0,0,"[(H1ms908a, Already here), (H1ms908a, walking)...",0 days 22:09:37.078011150,5.0,True


In [68]:
found_stop_times['intermediary_stops'].iloc[0]

[('H1ms908a', 'Already here'),
 ('H1ms908a', 'walking'),
 ('H1ms294c', '45183784-H_2024-H24_P2-Sem-N-3-11'),
 ('H1ms401a', 'walking'),
 ('H1ms401d', 'walking'),
 ('H1ms401d', '45184776-H_2024-H24_P2-Sem-N-3-11-0100000')]

In [74]:
found_stop_times['departure_time']=found_stop_times['departure_time'].apply(lambda x: x.total_seconds()/60)
found_stop_times['final_arrival_time']=found_stop_times['final_arrival_time'].apply(lambda x: x.total_seconds()/60)
found_stop_times['arrival_time']=found_stop_times['arrival_time'].apply(lambda x: x.total_seconds()/60)

In [75]:
proj_geo_stops=geo_stops.to_crs(4326)
m=geo_stops.merge(found_stop_times, how='right').sort_values('final_arrival_time').explore(column='explored')
best_path_gdf.explore(column='walking', m=m, cmap="spring")